In [1]:
# check GPU
!nvidia-smi || true
import torch
print("torch.cuda:", torch.cuda.is_available())


Sat Nov 15 21:15:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:

!pip install -q datasets sentence-transformers faiss-cpu transformers accelerate


# mount drive to persist indices and metas
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/rag_humaneval


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 48.4 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
from datasets import load_dataset
ds = load_dataset("openai_humaneval", split="test")
print("samples:", len(ds))
#
sample = ds[0]
print(sample.keys())
print(sample["task_id"])
print(sample["prompt"][:400])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

openai_humaneval/test-00000-of-00001.par(…):   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

samples: 164
dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'])
HumanEval/0
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



In [4]:
entries = []
for item in ds:
    entries.append({
        "task_id": item.get("task_id"),
        "prompt": item.get("prompt"),
        "canonical_solution": item.get("canonical_solution"),
        "test": item.get("test")
    })
len(entries)


164

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle
import os

MODEL_EMBED = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(MODEL_EMBED)

prompts = [ (e["prompt"] or "") for e in entries ]

EMB_PATH = "/content/drive/MyDrive/rag_humaneval/prompts_embeddings.npy"
META_PATH = "/content/drive/MyDrive/rag_humaneval/metas.pkl"

if not os.path.exists(EMB_PATH):
    embs = embedder.encode(prompts, convert_to_numpy=True, show_progress_bar=True, batch_size=64)
    # normalize for cosine with inner product
    embs = embs.astype("float32")
    embs = embs / (np.linalg.norm(embs, axis=1, keepdims=True) + 1e-10)
    np.save(EMB_PATH, embs)
    with open(META_PATH, "wb") as f:
        pickle.dump(entries, f)
    print("Saved embeddings and metas to Drive.")
else:
    embs = np.load(EMB_PATH)
    with open(META_PATH, "rb") as f:
        entries = pickle.load(f)
    print("Loaded embeddings and metas from Drive.")

print("emb shape:", embs.shape)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded embeddings and metas from Drive.
emb shape: (164, 384)


In [6]:
import faiss

faiss_index_path = "/content/drive/MyDrive/rag_humaneval/humaneval_index.faiss"

dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embs)
faiss.write_index(index, faiss_index_path)
print("FAISS index saved to Drive:", faiss_index_path)


FAISS index saved to Drive: /content/drive/MyDrive/rag_humaneval/humaneval_index.faiss


In [7]:
def retrieve(query, k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    q_emb = q_emb / (np.linalg.norm(q_emb, axis=1, keepdims=True) + 1e-10)
    D, I = index.search(q_emb.astype("float32"), k)
    results = []
    for score, idx in zip(D[0], I[0]):
        meta = dict(entries[idx])
        meta["score"] = float(score)
        results.append(meta)
    return results


q_res = retrieve("Return True if list contains two floats whose difference is less than threshold", k=3)
for r in q_res:
    print(r["task_id"], r["score"])
    print(r["prompt"][:200].replace("\n"," ") + "...")
    print("-----")


HumanEval/0 0.6516801714897156
from typing import List   def has_close_elements(numbers: List[float], threshold: float) -> bool:     """ Check if in given list of numbers, are any two numbers closer to each other than     given thr...
-----
HumanEval/52 0.6374454498291016
  def below_threshold(l: list, t: int):     """Return True if all numbers in the list l are below threshold t.     >>> below_threshold([1, 2, 4, 10], 100)     True     >>> below_threshold([1, 20, 4, 1...
-----
HumanEval/137 0.5076446533203125
 def compare_one(a, b):     """     Create a function that takes integers, floats, or strings representing     real numbers, and returns the larger variable in its given variable type.     Return None...
-----


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

GEN_MODEL = "Salesforce/codegen-350M-multi"
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(GEN_MODEL)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def build_prompt(user_prompt, retrieved):
    parts = []
    parts.append("### Task\n")
    parts.append(user_prompt.strip() + "\n\n")
    if retrieved:
        parts.append("### Retrieved examples:\n")
        for i, r in enumerate(retrieved):
            parts.append(f"# Example {i+1}\n")
            parts.append("Prompt:\n")
            parts.append((r.get("prompt") or "").strip() + "\n")
            parts.append("Solution:\n")
            sol = r.get("canonical_solution") or ""
            parts.append("```python\n" + sol.strip() + "\n```\n\n")
    parts.append("### Generate only the Python function implementation for the Task above.\n")
    return "\n".join(parts)

def generate_code_local(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    outs = model.generate(inputs, max_new_tokens=max_new_tokens, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(outs[0], skip_special_tokens=True)
    # return completion only if prompt prefix exists
    if decoded.startswith(prompt):
        return decoded[len(prompt):].strip()
    return decoded.strip()


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (

In [9]:
from datasets import load_dataset


dataset = load_dataset("openai_humaneval")

print("Total", len(dataset["test"]))


model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

Total 164


In [10]:
import textwrap
import tempfile

task = dataset["test"][0]
prompt = task["prompt"]
tests = task["test"]

#
generated_code = """
for idx, elem in enumerate(numbers):
    for idx2, elem2 in enumerate(numbers):
        if idx != idx2:
            distance = abs(elem - elem2)
            if distance < threshold:
                return True
return False
"""


full_code = prompt + textwrap.dedent(generated_code)


with tempfile.NamedTemporaryFile(delete=False, suffix=".py", mode="w") as tmp_file:
    tmp_file.write(full_code)
    file_path = tmp_file.name

print(" Code saved to:", file_path)
print("Saved code ")
print(full_code)


 Code saved to: /tmp/tmp_30cz5ns.py
Saved code 
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

for idx, elem in enumerate(numbers):
    for idx2, elem2 in enumerate(numbers):
        if idx != idx2:
            distance = abs(elem - elem2)
            if distance < threshold:
                return True
return False



In [11]:
import re

def extract_code(generated_text):
    m = re.search(r"```(?:python)?\n(.+?)```", generated_text, flags=re.S)
    if m:
        return m.group(1).strip()
    i = generated_text.find("def ")
    if i != -1:
        return generated_text[i:].strip()
    return generated_text.strip()


In [12]:
import subprocess, sys, tempfile, os, traceback, time

def parse_fn_name_from_prompt(prompt):
    m = re.search(r"def\s+([A-Za-z_]\w*)\s*\(", prompt)
    if m:
        return m.group(1)
    return None

def evaluate_generated_code(generated_code, prompt, test_code, timeout=8):
    fn_name = parse_fn_name_from_prompt(prompt)
    if not fn_name:
        m = re.search(r"def\s+([A-Za-z_]\w*)\s*\(", generated_code)
        fn_name = m.group(1) if m else None
    if not fn_name:
        return {"status":"no_function_name"}
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".py", mode="w", encoding="utf-8")
    try:
        tmp.write("# generated\n")
        tmp.write(generated_code + "\n\n")
        tmp.write("# test\n")
        tmp.write(test_code + "\n\n")
        tmp.write("try:\n")
        tmp.write(f"    fn = globals().get('{fn_name}')\n")
        tmp.write("    if fn is None:\n")
        tmp.write("        print('NO_FUNCTION')\n")
        tmp.write("        sys.exit(2)\n")
        tmp.write("    check(fn)\n")
        tmp.write("    print('PASS')\n")
        tmp.write("except Exception as e:\n")
        tmp.write("    print('FAIL')\n")
        tmp.write("    import traceback\n")
        tmp.write("    traceback.print_exc()\n")
        tmp.close()
        proc = subprocess.run([sys.executable, tmp.name], capture_output=True, text=True, timeout=timeout)
        out = proc.stdout + proc.stderr
        return {"status":"ran", "returncode":proc.returncode, "output":out}
    except subprocess.TimeoutExpired:
        return {"status":"timeout"}
    finally:
        try:
            os.unlink(tmp.name)
        except Exception:
            pass


In [13]:

idx = 0
meta = entries[idx]
user_prompt = meta["prompt"]
test_code = meta.get("test")


retr = retrieve(user_prompt, k=3)


prompt = build_prompt(user_prompt, retr)
generated_raw = generate_code_local(prompt, max_new_tokens=256)
code = extract_code(generated_raw)
print("=== Generated code ===")
print(code)


res = evaluate_generated_code(code, user_prompt, test_code, timeout=8)
print("Eval result:", res)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


=== Generated code ===
for idx, elem in enumerate(numbers):
          for idx2, elem2 in enumerate(numbers):
              if idx != idx2:
                  distance = abs(elem - elem2)
                  if distance < threshold:
                      return True

      return False
Eval result: {'status': 'ran', 'returncode': 1, 'output': '  File "/tmp/tmpvxlo_740.py", line 9\n    return False\n                ^\nIndentationError: unindent does not match any outer indentation level\n'}
